In [11]:
neki = 100
limit = 100

if neki and neki > limit:
    print(neki)

In [16]:
from tensorflow.python.compiler.tensorrt.trt_convert import TrtPrecisionMode
precisionMode = TrtPrecisionMode.INT8
print(type(precisionMode))


<class 'str'>


In [1]:
# Benchmark
import time
#import chess
import cppchess as chess
from game import Game
from mcts.c import run_mcts
from train.config import TrainingConfig
from trt_funcs import load_trt_checkpoint_latest

trt_func, model = load_trt_checkpoint_latest()

2024-05-23 11:13:35.511064: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 11:13:36.947640: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5831 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5
2024-05-23 11:13:37.572225: E tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:87] DefaultLogger 3: [runtime.cpp::~Runtime::346] Error Code 3: API Usage Error (Parameter check failed at: runtime/rt/runtime.cpp::~Runtime::346, condition: mEngineCounter.use_count() == 1. Destroying a runtime before destroying deserialized engines created by the runtime leads to undefined behavior.
)


In [2]:
game = Game(chess.Board(), resignable=True)
config = TrainingConfig()

times = []
for i in range(10):
    t1 = time.time()
    move, root, _ = run_mcts(
        game,
        config,
        trt_func, 
        num_simulations=10000, 
        time_limit=None,
        #verbose_move=True,
        engine_play=True)
    t2 = time.time()
    times.append(t2 - t1)

print(f"Average time: {sum(times) / len(times)}")

Average time: 2.609641122817993


In [3]:
# Calculate average nodes per second
nps = 10000 / (sum(times) / len(times))
nps

3831.944520096161

In [15]:
import chess
import chess.syzygy
import time
import numpy as np

tb = chess.syzygy.open_tablebase("/home/tomaz/syzygy")
if tb is None:
    print("Failed to open tablebase")
times = []
for i in range(10):
    t1 = time.time()
    board = chess.Board()
    wdl = tb.get_wdl(board)
    t2 = time.time()
    times.append(t2 - t1)

print(f"Average time: {sum(times) / len(times)}")

Average time: 0.00011489391326904296


In [14]:
for i in range(10):
    t1 = time.time()
    with chess.syzygy.open_tablebase("/home/tomaz/syzygy", load_dtz=False) as tablebase:
        board = chess.Board()
        wdl = tb.get_wdl(board)
    t2 = time.time()
    times.append(t2 - t1)

print(f"Average time: {sum(times) / len(times)}")


Average time: 0.003620326519012451


In [ ]:
""" if config.wdl_termination_move_limit and game.history_len >= config.wdl_termination_move_limit and game.board.halfmove_clock == 0:
    pieces_count = sum([1 for square in range(0, 64) if game.board.piece_at(square)])
    if pieces_count <= 5:
        with syzygy.open_tablebase(config.syzygy_tb_dir, load_dtz=False) as tablebase:
            board = pychess.Board(game.board.fen())
            wdl = tablebase.get_wdl(board)
            if wdl is not None and not (wdl == 1 or wdl == -1):
                if wdl == 0:
                    game.terminate(None, 8) # 8 -> tablebase outcome
                else:
                    game.terminate(game.to_play() if wdl > 0 else not game.to_play(), 8)
                break """